#### Note: 

 This section was originally written to play around with the data from a single participant. 

In [ ]:
import os, sys

print(sys.executable) # This is where this notebook is running in
print(sys.path) # This is where the downloaded modules are - sys.executable and sys.path need to be the same to be able to load the module.

# Solution:
# Run this in Terminal: (output of sys.executable) -m pip install (package name)

In [319]:
# Load packages to convert formats
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sn
import pingouin
from sklearn import preprocessing
import numpy as np
from numpy.core import einsumfunc
import utility as ut
from nilearn import signal

In [ ]:
# File where the NMA fMRI data are
path_data = "/Volumes/DPhil_Jelka/fmri_data/hcp_task"

try: 
    os.chdir(path_data) 
except OSError as error: 
    print(error)  

In [ ]:
# Check if you successfully changed directory
os.getcwd() 

In [ ]:
# Okay, you want hcp_task to be your specific directory for now
HCP_DIR = "/Volumes/DPhil_Jelka/fmri_data/hcp_task"
if not os.path.isdir(HCP_DIR):
    os.mkdir(HCP_DIR)

# The data shared for NMA projects is a subset of the full HCP dataset (0-338)
N_SUBJECTS = 339

# The data have already been aggregated into ROIs from the Glasser parcellation
N_PARCELS = 360

# The acquisition parameters for all tasks were identical
TR = 0.72  # Time resolution, in seconds

# The parcels are matched across hemispheres with the same order
HEMIS = ["Right", "Left"]

# Each experiment was repeated twice in each subject
N_RUNS = 2

# There are 7 tasks. Each has a number of 'conditions'

EXPERIMENTS = {
    'MOTOR'      : {'runs': [5,6],   'cond':['lf','rf','lh','rh','t','cue']},
    'WM'         : {'runs': [7,8],   'cond':['0bk_body','0bk_faces','0bk_places','0bk_tools','2bk_body','2bk_faces','2bk_places','2bk_tools']},
    'EMOTION'    : {'runs': [9,10],  'cond':['fear','neut']},
    'GAMBLING'   : {'runs': [11,12], 'cond':['loss','win']},
    'LANGUAGE'   : {'runs': [13,14], 'cond':['math','story']},
    'RELATIONAL' : {'runs': [15,16], 'cond':['match','relation']},
    'SOCIAL'     : {'runs': [17,18], 'cond':['mental','rnd']}
}

# You may want to limit the subjects used during code development. This will use all subjects.

In [ ]:
# Function to load one .npy timeseries file

def load_single_timeseries(subject, experiment, run, remove_mean=True):
    """
    Load timeseries data for a single subject and single run.
  
    Args:
    subject (int):      0-based subject ID to load
    experiment (str):   Name of experiment 
    run (int):          0-based run index, across all tasks
    remove_mean (bool): If True, subtract the parcel-wise mean (typically the mean BOLD signal is not of interest)

    Returns
    ts (n_parcel x n_timepoint array): Array of BOLD data values

    """
    bold_run  = EXPERIMENTS[experiment]['runs'][run]
    bold_path = f"{HCP_DIR}/subjects/{subject}/timeseries"
    bold_file = f"bold{bold_run}_Atlas_MSMAll_Glasser360Cortical.npy"
    ts = np.load(f"{bold_path}/{bold_file}")
    if remove_mean:
        ts -= ts.mean(axis=1, keepdims=True)
    return ts


def load_evs(subject, experiment, run):
    """Load EVs (explanatory variables) data for one task experiment.

    Args:
    subject (int): 0-based subject ID to load
    experiment (str) : Name of experiment

    Returns
    evs (list of lists): A list of frames associated with each condition

    """
    frames_list = []
    task_key = 'tfMRI_'+ experiment + '_'+['RL','LR'][run]
    for cond in EXPERIMENTS[experiment]['cond']:    
        ev_file  = f"{HCP_DIR}/subjects/{subject}/EVs/{task_key}/{cond}.txt"
        ev_array = np.loadtxt(ev_file, ndmin=2, unpack=True)
        ev       = dict(zip(["onset", "duration", "amplitude"], ev_array))
    
    # Determine when trial starts, rounded down
        start = np.floor(ev["onset"] / TR).astype(int)
    
    # Use trial duration to determine how many frames to include for trial
        duration = np.ceil(ev["duration"] / TR).astype(int)
    
    # Take the range of frames that correspond to this specific trial
        frames = [s + np.arange(0, d) for s, d in zip(start, duration)]
        frames_list.append(frames)
    
    return frames_list

In [ ]:
my_exp  = 'WM'
my_subj = 2
my_run  = 0

data_example = load_single_timeseries(subject = my_subj, experiment = my_exp, run = my_run, remove_mean = True)

print(data_example.shape) # 360 ROIs, 405 timepoints
print(data_example) # np.ndarray

In [ ]:
# Info about regions
regions = np.load(f"{HCP_DIR}/regions.npy").T
region_info = dict(
    name=regions[0].tolist(),
    network=regions[1],
    hemi=['Right']*int(N_PARCELS/2) + ['Left']*int(N_PARCELS/2),
)

In [ ]:
# We want ROIs in columns
regions_T = np.transpose(regions)

In [327]:
# Check how many networks are there
from collections import Counter

# Function to get unique values
def unique(list1):
   
    # Print directly by using * symbol
    print(*Counter(list1))

unique(regions[1]) #12 networks

Visual1 Visual2 Somatomotor Cingulo-Oper Language Default Frontopariet Auditory Dorsal-atten Posterior-Mu Orbito-Affec Ventral-Mult


In [ ]:
# Load explanatory variables - No clue what they current mean
evs = load_evs(subject = my_subj, experiment = my_exp, run = my_run)

In [ ]:
# Turn regions into a pd.dataframe
df_regions = pd.DataFrame(regions_T, columns = ['ROI', 'Network', 'Hemi'])
print(df_regions)

In [ ]:
# See how many regions you have for each network
df_regions['Network'].value_counts() # Not even close to an equal number

In [ ]:
# Again, we want ROIs in columns
data_example_T = np.transpose(data_example)
data_example_df = pd.DataFrame(data_example_T, columns = [df_regions['ROI'], df_regions['Network']])
print(data_example_df.shape) #405 time points, 360 ROIs

In [ ]:
# Transform original data_example to a pd.DataFrame
data_example_df_original = pd.DataFrame(data_example)
print(data_example_df_original)

In [ ]:
# Merge ROIs and networks
Regions_Neural = df_regions.merge(data_example_df_original, left_index=True, right_index=True)

In [ ]:
Regions_Neural.head()

In [ ]:
index_Network = np.arange(0, 360)
print(index_Network)

In [ ]:
columns_Network = Regions_Neural.columns
print(columns_Network)

In [ ]:
# Uncorrected full correlations between ROIs
ROI_CorrMatrix_Full = data_example_df.corr()
print(ROI_CorrMatrix_Full)

In [ ]:
# Heatmap - Didn't run, matrix is too big
sn.heatmap(ROI_CorrMatrix_Full, annot=True)
plt.show()

In [ ]:
# Uncorrected partial correlations between ROIs
ROI_CorrMatrix_Partial = data_example_df.pcorr()
print(ROI_CorrMatrix_Partial)

In [311]:
# The following procedure normalizes the response within ROIs over time
scaler = preprocessing.StandardScaler().fit(data_example_df)
data_example_df_zscore = scaler.transform(data_example_df)
data_example_df_zscore = pd.DataFrame(data_example_df_zscore, columns = data_example_df.columns)

In [313]:
print(data_example_df)

ROI            R_V1       R_MST        R_V6        R_V2        R_V3    R_V4  \
Network     Visual1     Visual2     Visual2     Visual2     Visual2 Visual2   
0          9.896543  127.426716   29.507259   30.120247   -3.957531  -27.34   
1        -45.703457   40.916716  -18.742741  -34.179753  -37.257531  -36.14   
2       -100.303457    4.466716  -35.672741  -68.579753  -61.157531  -67.34   
3       -140.403457   14.216716 -101.732741  -82.879753  -86.857531 -103.64   
4       -124.403457   64.766716   -5.712741  -66.279753  -61.657531  -41.34   
..              ...         ...         ...         ...         ...     ...   
400     -108.803457  -75.883284   -5.552741  -84.379753 -125.357531 -130.14   
401     -112.203457  -50.523284  -19.512741 -105.279753 -126.757531 -161.74   
402      -82.003457  -81.683284    7.547259  -76.679753 -129.757531 -159.44   
403      -71.803457  -29.553284   82.277259  -70.979753 -107.657531 -142.54   
404     -104.703457  -37.923284   68.857259  -60.879

In [ ]:
# Sanity check - Mean
np.mean(data_example_df_zscore['R_4'])

In [ ]:
# Sanity check - SD
np.std(data_example_df_zscore['R_4'])

#### Note:

Data for all participants and tasks (0-back and 2-back) are downloaded, pre-processed and analysed from here.

In [ ]:
# Add modules folder to Python's search path
from os import times
import sys
from pathlib import Path
from os.path import dirname, realpath, abspath
script_dir = Path(abspath(''))
module_dir = str(script_dir)
sys.path.insert(0, module_dir + '/modules')
print(module_dir)
print(script_dir)

In [ ]:
import task
db_path = '{}/data/hcp_task'.format(module_dir)
group = task.Group(db_path)

In [ ]:
# Load 0-back data - X are the features, y are the labels
X_0back, y_0back = group.extract_cons(['0bk_faces', '0bk_tools', '0bk_places', '0bk_body'])

In [ ]:
# Sanity check
print(X_0back.shape, y_0back.shape)

In [ ]:
# Load 2-back data
X_2back, y_2back = group.extract_cons(['2bk_faces', '2bk_tools', '2bk_places', '2bk_body'])

In [ ]:
print(X_2back.shape, y_2back.shape)

# 28080 - 360 (number of ROIs) x 78 (length of the time series)
# 1356 - 339 (number of subjects) x 4 (number of conditions)

#### Note:

Ignore the section below.

In [ ]:
# Create separate arrays with ROI names and time stamps
ROIs = np.array(df_regions['ROI'])
ROIs_Full = pd.DataFrame(np.repeat(ROIs, 78), columns = ['ROI'])

Timepoints = np.array(range(1,79))
Timepoints_Full = pd.DataFrame(np.tile(Timepoints, 360), columns = ['Time'])

In [ ]:
# Merge the two and create a combined column
ROIs_t = ROIs_Full.merge(Timepoints_Full, left_index=True, right_index=True)
ROIs_t['Time'] = ROIs_t['Time'].astype(str)
ROIs_t['ROI_t'] = ROIs_t['ROI'].str.cat(ROIs_t['Time'], sep ='_')

In [ ]:
# Sanity check
print(ROIs_t)

In [ ]:
# Convert the features space to pd.DataFrame
X_0back_df = pd.DataFrame(X_0back, columns = [ROIs_t['ROI_t']])
print(X_0back_df)

In [ ]:
# Add the following column: Participant 
X_0back_df['Participant'] = np.repeat(np.array(range(0,339)), 4)

In [ ]:
# Add the following column: Condition
X_0back_df['Condition'] = pd.Series(np.tile(np.array(['faces', 'tools', 'places', 'body']), 339))

In [ ]:
# Sanity check
print(X_0back_df.head())
X_0back_df.dtypes

In [ ]:
# Test if you can run classifiers in this data structure
from sklearn.linear_model import LogisticRegressionCV

# GLM
Classification = LogisticRegressionCV(cv = 8, random_state = 0, multi_class = 'multinomial', max_iter = 10000).fit(X_0back_df.iloc[:, 0:28], X_0back_df.iloc[:, 28081])

In [ ]:
# SVM
from sklearn import svm

svm.SVC().fit(X_0back_df.iloc[:, 0:28], X_0back_df.iloc[:, 28081])

#### Note:

Functional code for reshaping the dataframe can be found below.

In [ ]:
rois = regions[0]
subjects = np.arange(339)
conditions_0back = ['faces', 'tools', 'places', 'body']
conditions_2back = ['faces', 'tools', 'places', 'body']

In [ ]:
# Reshape the dataframe
index_0back = []
for cond in conditions_0back:
    for subj in subjects:
        for roi in rois:
            index_0back.append((cond, subj, roi))
len(index_0back)

In [ ]:
index_2back = []
for cond in conditions_2back:
    for subj in subjects:
        for roi in rois:
            index_2back.append((cond, subj, roi))
len(index_2back)

In [ ]:
# Reshape the 0-back task dataframe
X_new_0back = X_0back.reshape(488160, 78)

index_0back = pd.MultiIndex.from_tuples(index_0back)
X_df_0back = pd.DataFrame(X_new_0back, index = index_0back)
X_df_0back.index.names = ['Condition', 'Subject_id', 'ROI']

In [ ]:
# Sanity check
print(X_df_0back.head())
print(X_df_0back.shape)

In [ ]:
# Reshape the 2-back task dataframe
X_new_2back = X_2back.reshape(488160, 78)

index_2back = pd.MultiIndex.from_tuples(index_2back)
X_df_2back = pd.DataFrame(X_new_2back, index = index_2back)
X_df_2back.index.names = ['Condition', 'Subject_id', 'ROI']

In [ ]:
# Sanity check
print(X_df_2back.head())
print(X_df_2back.shape)

In [ ]:
# Run to unpack indices (to convert them to columns) - 0-back
X_df_0back.reset_index(level=0, inplace=True)
X_df_0back.reset_index(level=0, inplace=True)
X_df_0back.reset_index(level=0, inplace=True)

In [ ]:
# Run to unpack indices (to convert them to columns) - 2-back
X_df_2back.reset_index(level=0, inplace=True)
X_df_2back.reset_index(level=0, inplace=True)
X_df_2back.reset_index(level=0, inplace=True)

In [ ]:
# Add the network column
df_regions.reset_index(level=0, inplace=True)
print(df_regions)

In [ ]:
# Sanity check
print(X_df_0back.head())
print(X_df_0back.shape)

In [367]:
# Adding the information about the brain network to the data frame
X_0back_full = X_df_0back.merge(df_regions, left_on='ROI', right_on='ROI')
X_2back_full = X_df_2back.merge(df_regions, left_on='ROI', right_on='ROI')

In [372]:
# Sanity check
X_0back_full.head()

,,,,0,1,2,3,4,5,6,7,8,9,...,68,69,70,71,72,73,74,75,76,77
Subject_id,Condition,ROI,Network,,,,,,,,,,,,,,,,,,,,,
0,faces,R_V1,Visual1,12028.00,11983.20,11991.30,11979.20,12006.00,11998.1,12008.00,12009.80,11986.20,12008.80,...,12078.00,12048.50,12029.40,12021.90,12003.70,11981.90,12006.80,11996.90,12034.00,12051.10
1,faces,R_V1,Visual1,9663.47,9690.52,9682.68,9653.18,9666.53,9698.1,9670.03,9687.82,9686.86,9682.41,...,9658.34,9657.14,9680.39,9680.88,9668.43,9641.42,9651.17,9621.19,9628.23,9635.76
2,faces,R_V1,Visual1,10948.80,10945.20,10931.50,10923.60,10973.60,10961.0,10985.00,11034.40,11041.10,11017.50,...,10960.40,10959.90,10949.90,10986.70,10985.20,11004.30,10956.50,10963.70,10980.10,10975.30
3,faces,R_V1,Visual1,11508.60,11518.10,11465.00,11477.70,11469.70,11524.6,11527.30,11554.60,11541.30,11532.20,...,11670.00,11690.20,11717.20,11719.50,11737.20,11684.80,11738.30,11735.60,11705.20,11721.20
4,faces,R_V1,Visual1,11814.20,11780.40,11793.60,11780.60,11783.40,11801.0,11841.90,11813.70,11858.60,11876.90,...,11372.60,11375.90,11425.90,11412.90,11448.60,11434.10,11434.60,11371.30,11365.40,11371.70


In [373]:
# Sanity check
X_2back_full.head()

,,,,0,1,2,3,4,5,6,7,8,9,...,68,69,70,71,72,73,74,75,76,77
Subject_id,Condition,ROI,Network,,,,,,,,,,,,,,,,,,,,,
0,faces,R_V1,Visual1,12004.50,12000.50,12004.10,12000.50,12023.50,12060.80,12069.90,12122.10,12146.9,12136.20,...,12172.40,12146.10,12176.30,12157.90,12134.10,12170.20,12142.10,12151.80,12126.50,12158.70
1,faces,R_V1,Visual1,9667.16,9649.17,9678.08,9662.16,9649.98,9615.69,9627.43,9630.69,9628.2,9646.23,...,9721.84,9712.71,9746.95,9724.27,9738.19,9735.02,9716.98,9751.03,9728.04,9706.46
2,faces,R_V1,Visual1,10919.50,10912.50,10885.30,10913.40,10922.50,10946.30,10940.10,11005.80,11056.9,11062.60,...,11028.10,11018.00,11054.10,11020.00,11024.20,11025.00,11014.30,10997.30,10999.80,11018.60
3,faces,R_V1,Visual1,11495.60,11472.30,11534.30,11518.80,11514.30,11561.60,11562.70,11532.20,11575.7,11533.50,...,11748.10,11740.70,11770.10,11825.60,11812.40,11807.60,11804.40,11757.80,11750.10,11763.40
4,faces,R_V1,Visual1,11726.70,11802.10,11779.90,11769.50,11790.20,11787.40,11832.50,11795.80,11811.9,11818.60,...,11353.20,11352.00,11379.30,11352.20,11371.60,11327.40,11346.90,11340.70,11344.00,11368.90


In [370]:
# Set indices
X_0back_full = X_0back_full.set_index(['Subject_id', 'Condition', 'ROI', 'Network'])
X_2back_full = X_2back_full.set_index(['Subject_id', 'Condition', 'ROI', 'Network'])

In [371]:
# Delete unnecessary columns
del X_0back_full["index"]
del X_0back_full["Hemi"]
del X_2back_full["index"]
del X_2back_full["Hemi"]

In [374]:
X_2back_0back_contr_ROI = X_2back_full - X_0back_full

In [375]:
# Sanity check
X_2back_0back_contr_ROI.head()

,,,,0,1,2,3,4,5,6,7,8,9,...,68,69,70,71,72,73,74,75,76,77
Subject_id,Condition,ROI,Network,,,,,,,,,,,,,,,,,,,,,
0,faces,R_V1,Visual1,-23.50,17.30,12.8,21.30,17.50,62.70,61.9,112.30,160.70,127.40,...,94.4,97.60,146.90,136.00,130.40,188.3,135.30,154.90,92.50,107.6
1,faces,R_V1,Visual1,3.69,-41.35,-4.6,8.98,-16.55,-82.41,-42.6,-57.13,-58.66,-36.18,...,63.5,55.57,66.56,43.39,69.76,93.6,65.81,129.84,99.81,70.7
2,faces,R_V1,Visual1,-29.30,-32.70,-46.2,-10.20,-51.10,-14.70,-44.9,-28.60,15.80,45.10,...,67.7,58.10,104.20,33.30,39.00,20.7,57.80,33.60,19.70,43.3
3,faces,R_V1,Visual1,-13.00,-45.80,69.3,41.10,44.60,37.00,35.4,-22.40,34.40,1.30,...,78.1,50.50,52.90,106.10,75.20,122.8,66.10,22.20,44.90,42.2
4,faces,R_V1,Visual1,-87.50,21.70,-13.7,-11.10,6.80,-13.60,-9.4,-17.90,-46.70,-58.30,...,-19.4,-23.90,-46.60,-60.70,-77.00,-106.7,-87.70,-30.60,-21.40,-2.8


In [ ]:
X_0back_full_grouped = X_0back_full.groupby(['Subject_id', 'ROI'])
#scaler = preprocessing.StandardScaler().fit(X_0back_full_grouped.iloc[:, 3:81])
nilearn.signal.clean(X_0back_full.iloc[:, 3:81], runs=None, detrend=True, standardize='zscore') 
#data_example_df_zscore = scaler.transform(data_example_df)

#### Note: 

Calculate the mean activity on the network level.

In [ ]:
# Group by condition, network and participant and calculate the mean network activity at each of the 78 time steps
X_0back_full_net = X_0back_full.groupby(['Subject_id', 'Condition', 'Network']).mean()
X_2back_full_net = X_2back_full.groupby(['Subject_id', 'Condition', 'Network']).mean()

In [ ]:
# 339 (subj) x 4 (cond) x 12 (net) = 16272 rows
X_0back_full_net.shape

In [ ]:
# Sanity check
X_0back_full_net.head()

In [ ]:
# Return indices to columns (0-back)
X_0back_full_net.reset_index(level=0, inplace=True)
X_0back_full_net.reset_index(level=0, inplace=True)
X_0back_full_net.reset_index(level=0, inplace=True)

In [ ]:
# Return indices to columns (2-back)
X_2back_full_net.reset_index(level=0, inplace=True)
X_2back_full_net.reset_index(level=0, inplace=True)
X_2back_full_net.reset_index(level=0, inplace=True)

In [ ]:
# Sanity check
X_0back_full_net.head()

In [314]:
# Sanity check
X_2back_full_net.head()

,,,0,1,2,3,4,5,6,7,8,9,...,68,69,70,71,72,73,74,75,76,77
Network,Condition,Subject_id,,,,,,,,,,,,,,,,,,,,,
Auditory,body,0,10513.302667,10510.148000,10535.079333,10521.920000,10515.240667,10529.814667,10548.097333,10561.133333,10554.373333,10564.174000,...,10547.381333,10548.404667,10581.230667,10578.819333,10552.318000,10553.810000,10557.700000,10547.729333,10555.196000,10572.920667
Cingulo-Oper,body,0,11359.838393,11350.669643,11362.828036,11357.317143,11354.753036,11354.653036,11372.082679,11388.209643,11399.702857,11410.989464,...,11184.943571,11190.940714,11186.418750,11194.230893,11187.954643,11196.669643,11194.907500,11183.193929,11209.787500,11206.237500
Default,body,0,10280.175217,10277.216522,10268.863913,10280.523478,10267.522174,10282.300870,10301.117391,10311.949130,10314.795652,10310.440435,...,10348.343478,10356.838261,10364.836957,10358.145217,10355.730000,10362.967391,10361.825217,10340.093913,10340.578696,10358.893913
Dorsal-atten,body,0,10407.442857,10426.668571,10380.267143,10386.244286,10394.920000,10415.598571,10426.321429,10443.237143,10453.041429,10457.281429,...,10369.707143,10384.895714,10395.381429,10389.944286,10396.580000,10404.450000,10411.432857,10394.377143,10414.470000,10411.192857
Frontopariet,body,0,10888.065600,10873.332200,10855.890200,10869.319600,10884.365800,10897.929400,10929.934000,10956.846200,10962.209800,10965.212200,...,10657.163000,10687.228600,10686.270600,10684.482400,10675.743600,10698.277000,10689.713600,10680.966000,10680.650800,10679.955400


In [ ]:
# Delete the column you don't need
del X_0back_full_net["index"]
del X_2back_full_net["index"]

#### Note:

Ignore the code below - DO NOT rename the columns as that will throw an error when you try to subtract the 2-back and 0-back dataframes.

In [ ]:
# Rename time point columns
cols_0back = X_0back_full_net.columns
cols_2back = X_2back_full_net.columns

In [ ]:
cols_0back = np.asarray(cols_0back, dtype=str)
cols_2back = np.asarray(cols_2back, dtype=str)

In [ ]:
cols_0back = np.core.defchararray.add(cols_0back, '_0_back')
cols_2back = np.core.defchararray.add(cols_2back, '_2_back')

cols_0back[0] = 'Network'
cols_0back[1] = 'Condition'
cols_0back[2] = 'Subject_id'

cols_2back[0] = 'Network'
cols_2back[1] = 'Condition'
cols_2back[2] = 'Subject_id'

In [ ]:
# Sanity check
print(cols_0back)
print(cols_2back)

In [ ]:
# Change the column names
X_0back_full_net.columns = cols_0back
X_2back_full_net.columns = cols_2back

In [ ]:
# Sanity check
X_0back_full_net.head()

In [ ]:
# Sanity check
X_2back_full_net.head()

In [ ]:
# Merge 0-back and 2-back tasks
X_0back_2back = X_0back_full_net.merge(X_2back_full_net, left_on=['Network', 'Condition', 'Subject_id'], right_on=['Network', 'Condition', 'Subject_id'])

In [ ]:
# Sanity check
X_0back_2back.head(12)

#### Note:

Create a contrast dataframe.

In [ ]:
# Contrast 2-back - 0-back
X_0back_full_net = X_0back_full_net.set_index(['Network', 'Condition', 'Subject_id'])
X_2back_full_net = X_2back_full_net.set_index(['Network', 'Condition', 'Subject_id'])

In [ ]:
# Sanity check
X_0back_full_net.head()

In [ ]:
# Sanity check
X_2back_full_net.head()

In [ ]:
# Create a contrast dataframe
X_2back_0back_contr = X_2back_full_net - X_0back_full_net

In [322]:
X_2back_0back_contr.head()

,Subject_id,Condition,Network,0,1,2,3,4,5,6,...,68,69,70,71,72,73,74,75,76,77
0,0,body,Auditory,-57.608000,-49.334667,-22.963333,-36.051333,-53.830667,-46.014000,-49.491333,...,9.984000,15.039333,35.524000,41.800667,27.257333,-13.035333,-0.525333,-7.809333,-10.152000,24.150000
1,0,body,Cingulo-Oper,-44.773750,-47.277857,-42.373214,-55.605893,-52.908393,-56.814464,-37.060536,...,-6.824643,-6.127143,-7.855179,11.999821,13.727679,3.312857,-5.851607,-11.811786,10.299286,-5.914107
2,0,body,Default,-21.890870,-38.143913,-47.311304,-39.153043,-38.462609,-29.906522,-30.285652,...,15.991304,8.194348,23.814783,32.672609,18.934783,32.797391,16.326522,7.706522,-7.081739,18.633478
3,0,body,Dorsal-atten,-29.175714,-10.877143,-49.588571,-56.308571,-45.622857,-51.738571,-66.812857,...,25.592857,32.951429,11.541429,40.477143,34.551429,47.691429,52.791429,35.381429,73.208571,40.994286
4,0,body,Frontopariet,-88.432400,-103.538200,-123.485400,-120.376200,-108.184000,-103.527200,-84.803800,...,34.875000,48.201200,50.335200,64.581800,49.211000,77.063400,52.881600,51.460600,55.243600,34.334600


In [ ]:
# Return indices to columns (2-back)
X_2back_0back_contr.reset_index(level=0, inplace=True)
X_2back_0back_contr.reset_index(level=0, inplace=True)
X_2back_0back_contr.reset_index(level=0, inplace=True)

In [377]:
X_2back_0back_contr_ROI.reset_index(level=0, inplace=True)
X_2back_0back_contr_ROI.reset_index(level=0, inplace=True)
X_2back_0back_contr_ROI.reset_index(level=0, inplace=True)
X_2back_0back_contr_ROI.reset_index(level=0, inplace=True)

In [378]:
# Sanity check
X_2back_0back_contr_ROI.head()

,Network,ROI,Condition,Subject_id,0,1,2,3,4,5,...,68,69,70,71,72,73,74,75,76,77
0,Visual1,R_V1,faces,0,-23.50,17.30,12.8,21.30,17.50,62.70,...,94.4,97.60,146.90,136.00,130.40,188.3,135.30,154.90,92.50,107.6
1,Visual1,R_V1,faces,1,3.69,-41.35,-4.6,8.98,-16.55,-82.41,...,63.5,55.57,66.56,43.39,69.76,93.6,65.81,129.84,99.81,70.7
2,Visual1,R_V1,faces,2,-29.30,-32.70,-46.2,-10.20,-51.10,-14.70,...,67.7,58.10,104.20,33.30,39.00,20.7,57.80,33.60,19.70,43.3
3,Visual1,R_V1,faces,3,-13.00,-45.80,69.3,41.10,44.60,37.00,...,78.1,50.50,52.90,106.10,75.20,122.8,66.10,22.20,44.90,42.2
4,Visual1,R_V1,faces,4,-87.50,21.70,-13.7,-11.10,6.80,-13.60,...,-19.4,-23.90,-46.60,-60.70,-77.00,-106.7,-87.70,-30.60,-21.40,-2.8


In [379]:
# Split the networks - Original ROI activity
Visual1 = X_2back_0back_contr_ROI.loc[X_2back_0back_contr_ROI['Network'] == 'Visual1']
Visual2 = X_2back_0back_contr_ROI.loc[X_2back_0back_contr_ROI['Network'] == 'Visual2']
Somatomotor = X_2back_0back_contr_ROI.loc[X_2back_0back_contr_ROI['Network'] == 'Somatomotor']
Cingulo_Oper = X_2back_0back_contr_ROI.loc[X_2back_0back_contr_ROI['Network'] == 'Cingulo-Oper']
Language = X_2back_0back_contr_ROI.loc[X_2back_0back_contr_ROI['Network'] == 'Language']
Default = X_2back_0back_contr_ROI.loc[X_2back_0back_contr_ROI['Network'] == 'Default']
Frontopariet = X_2back_0back_contr_ROI.loc[X_2back_0back_contr_ROI['Network'] == 'Frontopariet']
Auditory = X_2back_0back_contr_ROI.loc[X_2back_0back_contr_ROI['Network'] == 'Auditory']
Dorsal_atten = X_2back_0back_contr_ROI.loc[X_2back_0back_contr_ROI['Network'] == 'Dorsal-atten']
Posterior_Mu = X_2back_0back_contr_ROI.loc[X_2back_0back_contr_ROI['Network'] == 'Posterior-Mu']
Orbito_Affec = X_2back_0back_contr_ROI.loc[X_2back_0back_contr_ROI['Network'] == 'Orbito-Affec']
Ventral_Mult = X_2back_0back_contr_ROI.loc[X_2back_0back_contr_ROI['Network'] == 'Ventral-Mult']

In [407]:
# Sanity check
Language.head(12)

,Network,ROI,Condition,Subject_id,0,1,2,3,4,5,...,68,69,70,71,72,73,74,75,76,77
13560,Language,R_PEF,faces,0,-116.97,-145.35,-211.31,-88.87,-71.69,-38.10,...,57.35,53.72,-5.46,113.20,79.83,28.75,114.11,55.75,56.36,89.29
13561,Language,R_PEF,faces,1,55.74,51.50,29.12,-11.74,-45.37,-20.55,...,5.88,12.27,88.07,171.27,76.67,122.36,141.72,97.73,75.06,120.79
13562,Language,R_PEF,faces,2,-51.00,-7.90,65.50,84.90,72.30,-16.20,...,98.06,55.40,141.83,147.14,154.61,156.08,125.07,170.27,117.57,111.49
13563,Language,R_PEF,faces,3,-150.07,-116.69,-90.25,-43.61,-63.91,-80.66,...,49.73,-56.07,-19.60,65.90,38.51,57.09,104.68,56.54,43.54,-37.42
13564,Language,R_PEF,faces,4,-125.10,-46.20,-44.10,-66.90,-162.40,-312.20,...,-95.90,-45.10,-264.90,-70.90,-86.50,-241.10,-130.90,-274.00,-265.00,-88.30
13565,Language,R_PEF,faces,5,-124.20,-94.30,-122.30,-31.50,-40.70,19.00,...,-166.40,-89.50,-105.60,-112.10,-39.10,-50.30,-17.90,-113.10,-114.40,-3.70
13566,Language,R_PEF,faces,6,20.68,185.95,120.70,139.97,202.23,229.62,...,-9.30,18.90,76.00,45.10,-18.80,31.70,-62.40,-82.40,-59.40,-101.30
13567,Language,R_PEF,faces,7,-88.80,-111.40,-183.00,20.10,92.50,40.10,...,-34.70,-123.80,-74.20,-22.30,-125.20,-31.00,-35.20,41.80,61.10,51.90
13568,Language,R_PEF,faces,8,24.75,112.90,119.63,8.42,29.74,22.15,...,58.51,23.95,-42.05,-71.51,24.11,-87.22,-98.28,22.17,-38.99,-102.63
13569,Language,R_PEF,faces,9,19.22,-14.64,111.27,5.48,-90.95,-59.04,...,-58.67,-150.24,-200.18,-201.03,-209.30,-199.98,-140.00,-154.82,-220.57,-175.98


In [382]:
# Sanity check
unique(Language['ROI'])
unique(Language['Network'])

R_PEF R_7PL R_MIP R_LIPd R_6a R_PFt R_AIP R_PHA3 R_TE2p R_PHT R_PGp R_IP0 L_7PL L_MIP L_LIPd L_6a L_PFt L_AIP L_PHA3 L_TE2p L_PHT L_PGp L_IP0
Language


In [357]:
# Split the networks - Mean activity
Visual1_mean = X_2back_0back_contr.loc[X_2back_0back_contr['Network'] == 'Visual1']
Visual2_mean = X_2back_0back_contr.loc[X_2back_0back_contr['Network'] == 'Visual2']
Somatomotor_mean = X_2back_0back_contr.loc[X_2back_0back_contr['Network'] == 'Somatomotor']
Cingulo_Oper_mean = X_2back_0back_contr.loc[X_2back_0back_contr['Network'] == 'Cingulo-Oper']
Language_mean = X_2back_0back_contr.loc[X_2back_0back_contr['Network'] == 'Language']
Default_mean = X_2back_0back_contr.loc[X_2back_0back_contr['Network'] == 'Default']
Frontopariet_mean = X_2back_0back_contr.loc[X_2back_0back_contr['Network'] == 'Frontopariet']
Auditory_mean = X_2back_0back_contr.loc[X_2back_0back_contr['Network'] == 'Auditory']
Dorsal_atten_mean = X_2back_0back_contr.loc[X_2back_0back_contr['Network'] == 'Dorsal-atten']
Posterior_Mu_mean = X_2back_0back_contr.loc[X_2back_0back_contr['Network'] == 'Posterior-Mu']
Orbito_Affec_mean = X_2back_0back_contr.loc[X_2back_0back_contr['Network'] == 'Orbito-Affec']
Ventral_Mult_mean = X_2back_0back_contr.loc[X_2back_0back_contr['Network'] == 'Ventral-Mult']

In [409]:
Language_mean.head(12)

,Subject_id,Condition,Network,0,1,2,3,4,5,6,...,68,69,70,71,72,73,74,75,76,77
5,0,body,Language,-25.437391,-23.242174,-28.704783,-69.928261,-65.927826,-62.439565,-52.968261,...,6.946087,47.531739,66.267391,77.748261,47.050000,61.791739,44.957391,53.844783,49.295652,36.824783
17,0,faces,Language,-7.507826,-7.934783,1.685217,2.546522,5.877391,13.499565,7.877826,...,72.237826,65.439130,74.092609,70.059565,80.077391,78.955217,100.646522,74.118696,57.241304,57.605652
29,0,places,Language,-47.940000,-34.703478,-12.952174,21.867826,0.742174,-11.567391,-26.173913,...,-4.670000,-8.290435,16.133913,17.693043,26.562609,53.039565,48.057826,15.251739,32.573043,28.733043
41,0,tools,Language,1.925217,-1.026522,40.670435,69.505217,88.901304,73.719565,26.661304,...,1.509130,12.185217,5.251304,-1.877826,3.597826,29.208696,63.970870,38.460435,51.836087,64.586957
53,1,body,Language,-1.032609,-11.955652,-32.775652,-23.463478,5.369130,12.230435,5.007391,...,36.098261,52.750870,46.104348,24.740000,11.137391,24.034348,-15.181304,-24.037391,-22.313043,-53.826957
65,1,faces,Language,-56.203913,-61.963478,-27.967391,-45.467391,-54.963913,-51.356957,-22.124348,...,45.710000,56.493043,69.831739,84.686957,97.709565,102.434348,109.809130,112.023478,76.320435,67.396087
77,1,places,Language,8.503913,39.978696,72.992609,53.173913,29.031739,10.039130,25.119565,...,52.690000,33.276087,-8.812174,-11.467826,1.845652,-16.437826,-8.824783,3.160435,-8.261739,-19.747826
89,1,tools,Language,70.267391,113.352609,111.053043,86.400870,63.474783,22.448696,40.840870,...,2.154348,19.077826,14.606087,46.499565,57.668261,73.246522,57.614783,65.172174,49.473043,81.720435
101,2,body,Language,-36.515652,-56.963913,-54.345652,-48.126522,-25.269565,-49.730000,-44.574348,...,23.308696,46.892174,62.223478,84.952174,75.093913,62.081304,32.139130,43.279565,51.056522,30.589130
113,2,faces,Language,-36.788696,-8.673913,-13.726957,-21.470000,-29.845652,-42.590000,-53.581739,...,61.529130,63.073478,73.813043,45.057391,38.294348,56.926957,65.999565,59.423043,56.843478,63.423913


#### NEXT STEPS:

In [ ]:
# TO DO:
# NORMALIZING DATA - It is very important to figure out what is meant by this and how to execute it.
# NORMALIZING DATA - This might probably need to be done before creating contrasts or anything.
# WITHIN-NETWORK CLASSIFICATION - Doing the classification within each network + Regularization?
# CROSS-VALIDATION - Are we 'manually' splitting the data into train/test or is the function doing that for us?
# REGULARIZATION

In [323]:
# sklearn imports
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression

#### Note:
Fitting the GLM

In [335]:
# Define the model - Same for all networks
model = LogisticRegression(multi_class = 'multinomial', max_iter = 10000, solver='lbfgs', penalty='l2', C=1.0)

# Define the model evaluation procedure - Same for all networks
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

#### Note: 

Network-by-network analysis (with ROI-level activity)

In [393]:
# Evaluate the model and collect the scores - Visual 1
n_scores_V1_ROI = cross_val_score(model, Visual1.iloc[:, 4:82], Visual1.iloc[:, 2], scoring='accuracy', cv=cv, n_jobs=-1)

# Report the model performance (M = 0.480, SD = 0.018)
print('Mean Accuracy: %.3f (%.3f)' % (np.mean(n_scores_V1_ROI), np.std(n_scores_V1_ROI)))

Mean Accuracy: 0.480 (0.018)


In [394]:
# Evaluate the model and collect the scores - Visual 2
n_scores_V2_ROI = cross_val_score(model, Visual2.iloc[:, 4:82], Visual2.iloc[:, 2], scoring='accuracy', cv=cv, n_jobs=-1)

# Report the model performance (M = 0.526, SD = 0.005)
print('Mean Accuracy: %.3f (%.3f)' % (np.mean(n_scores_V2_ROI), np.std(n_scores_V2_ROI)))

Mean Accuracy: 0.526 (0.005)


In [395]:
# Evaluate the model and collect the scores - Somatomotor
n_scores_SM_ROI = cross_val_score(model, Somatomotor.iloc[:, 4:82], Somatomotor.iloc[:, 2], scoring='accuracy', cv=cv, n_jobs=-1)

# Report the model performance (M = 0.404, SD = 0.007)
print('Mean Accuracy: %.3f (%.3f)' % (np.mean(n_scores_SM_ROI), np.std(n_scores_SM_ROI)))

Mean Accuracy: 0.404 (0.007)


In [396]:
# Evaluate the model and collect the scores - Cingulo-Oper
n_scores_CO_ROI = cross_val_score(model, Cingulo_Oper.iloc[:, 4:82], Cingulo_Oper.iloc[:, 2], scoring='accuracy', cv=cv, n_jobs=-1)

# Report the model performance (M = 0.412, SD = 0.005)
print('Mean Accuracy: %.3f (%.3f)' % (np.mean(n_scores_CO_ROI), np.std(n_scores_CO_ROI)))

Mean Accuracy: 0.412 (0.005)


In [398]:
# Evaluate the model and collect the scores - Language
n_scores_LA_ROI = cross_val_score(model, Language.iloc[:, 4:82], Language.iloc[:, 2], scoring='accuracy', cv=cv, n_jobs=-1)

# Report the model performance (M = 0.51, SD = 0.008)
print('Mean Accuracy: %.3f (%.3f)' % (np.mean(n_scores_LA_ROI), np.std(n_scores_LA_ROI)))

Mean Accuracy: 0.509 (0.008)


In [399]:
# Evaluate the model and collect the scores - Default
n_scores_DE_ROI = cross_val_score(model, Default.iloc[:, 4:82], Default.iloc[:, 2], scoring='accuracy', cv=cv, n_jobs=-1)

# Report the model performance (M = 0.424, SD = 0.009)
print('Mean Accuracy: %.3f (%.3f)' % (np.mean(n_scores_DE_ROI), np.std(n_scores_DE_ROI)))

Mean Accuracy: 0.424 (0.009)


In [401]:
# Evaluate the model and collect the scores - Frontopariet
n_scores_FP_ROI = cross_val_score(model, Frontopariet.iloc[:, 4:82], Frontopariet.iloc[:, 2], scoring='accuracy', cv=cv, n_jobs=-1)

# Report the model performance (M = 0.468, SD = 0.008)
print('Mean Accuracy: %.3f (%.3f)' % (np.mean(n_scores_FP_ROI), np.std(n_scores_FP_ROI)))

Mean Accuracy: 0.468 (0.008)


In [402]:
# Evaluate the model and collect the scores - Auditory
n_scores_AU_ROI = cross_val_score(model, Auditory.iloc[:, 4:82], Auditory.iloc[:, 2], scoring='accuracy', cv=cv, n_jobs=-1)

# Report the model performance (M = 0.459, SD = 0.010)
print('Mean Accuracy: %.3f (%.3f)' % (np.mean(n_scores_AU_ROI), np.std(n_scores_AU_ROI)))

Mean Accuracy: 0.459 (0.010)


In [403]:
# Evaluate the model and collect the scores - Dorsal-atten
n_scores_DA_ROI = cross_val_score(model, Dorsal_atten.iloc[:, 4:82], Dorsal_atten.iloc[:, 2], scoring='accuracy', cv=cv, n_jobs=-1)

# Report the model performance (M = 0.414, SD = 0.016)
print('Mean Accuracy: %.3f (%.3f)' % (np.mean(n_scores_DA_ROI), np.std(n_scores_DA_ROI)))

Mean Accuracy: 0.414 (0.016)


In [404]:
# Evaluate the model and collect the scores - Posterior-Mu
n_scores_PM_ROI = cross_val_score(model, Posterior_Mu.iloc[:, 4:82], Posterior_Mu.iloc[:, 2], scoring='accuracy', cv=cv, n_jobs=-1)

# Report the model performance (M = 0.397, SD = 0.004)
print('Mean Accuracy: %.3f (%.3f)' % (np.mean(n_scores_PM_ROI), np.std(n_scores_PM_ROI)))

Mean Accuracy: 0.397 (0.004)


In [405]:
# Evaluate the model and collect the scores - Orbito-Affec
n_scores_OA_ROI = cross_val_score(model, Orbito_Affec.iloc[:, 4:82], Orbito_Affec.iloc[:, 2], scoring='accuracy', cv=cv, n_jobs=-1)

# Report the model performance (M = 0.329, SD = 0.016)
print('Mean Accuracy: %.3f (%.3f)' % (np.mean(n_scores_OA_ROI), np.std(n_scores_OA_ROI)))

Mean Accuracy: 0.329 (0.016)


In [406]:
# Evaluate the model and collect the scores - Ventral-Mult
n_scores_VM_ROI = cross_val_score(model, Ventral_Mult.iloc[:, 4:82], Ventral_Mult.iloc[:, 2], scoring='accuracy', cv=cv, n_jobs=-1)

# Report the model performance (M = 0.423, SD = 0.018)
print('Mean Accuracy: %.3f (%.3f)' % (np.mean(n_scores_VM_ROI), np.std(n_scores_VM_ROI)))

Mean Accuracy: 0.423 (0.018)


#### Note: 

Network-by-network analysis (with mean network-level activity)

In [342]:
# Evaluate the model and collect the scores - Visual 1
n_scores_V1 = cross_val_score(model, Visual1_mean.iloc[:, 3:81], Visual1_mean.iloc[:, 1], scoring='accuracy', cv=cv, n_jobs=-1)

# Report the model performance (M = 0.51, SD = 0.034)
print('Mean Accuracy: %.3f (%.3f)' % (np.mean(n_scores_V1), np.std(n_scores_V1)))

Mean Accuracy: 0.507 (0.034)


In [343]:
# Evaluate the model and collect the scores - Visual 2
n_scores_V2 = cross_val_score(model, Visual2_mean.iloc[:, 3:81], Visual2_mean.iloc[:, 1], scoring='accuracy', cv=cv, n_jobs=-1)

# Report the model performance (M = 0.65, SD = 0.040)
print('Mean Accuracy: %.3f (%.3f)' % (np.mean(n_scores_V2), np.std(n_scores_V2)))

Mean Accuracy: 0.653 (0.040)


In [344]:
# Evaluate the model and collect the scores - Somatomotor
n_scores_SM = cross_val_score(model, Somatomotor_mean.iloc[:, 3:81], Somatomotor_mean.iloc[:, 1], scoring='accuracy', cv=cv, n_jobs=-1)

# Eeport the model performance (M = 0.44, SD = 0.041)
print('Mean Accuracy: %.3f (%.3f)' % (np.mean(n_scores_SM), np.std(n_scores_SM)))

Mean Accuracy: 0.437 (0.041)


In [345]:
# Evaluate the model and collect the scores - Cingulo-Oper
n_scores_CO = cross_val_score(model, Cingulo_Oper_mean.iloc[:, 3:81], Cingulo_Oper_mean.iloc[:, 1], scoring='accuracy', cv=cv, n_jobs=-1)

# Report the model performance (M = 0.47, SD = 0.039)
print('Mean Accuracy: %.3f (%.3f)' % (np.mean(n_scores_CO), np.std(n_scores_CO)))

Mean Accuracy: 0.474 (0.039)


In [346]:
# Evaluate the model and collect the scores - Language
n_scores_LA = cross_val_score(model, Language_mean.iloc[:, 3:81], Language_mean.iloc[:, 1], scoring='accuracy', cv=cv, n_jobs=-1)

# Report the model performance (M = 0.61, SD = 0.043)
print('Mean Accuracy: %.3f (%.3f)' % (np.mean(n_scores_LA), np.std(n_scores_LA)))

Mean Accuracy: 0.605 (0.043)


In [347]:
# Evaluate the model and collect the scores - Default
n_scores_DE = cross_val_score(model, Default_mean.iloc[:, 3:81], Default_mean.iloc[:, 1], scoring='accuracy', cv=cv, n_jobs=-1)

# Report the model performance (M = 0.49, SD = 0.039)
print('Mean Accuracy: %.3f (%.3f)' % (np.mean(n_scores_DE), np.std(n_scores_DE)))

Mean Accuracy: 0.492 (0.039)


In [348]:
# Evaluate the model and collect the scores - Frontopariet
n_scores_FP = cross_val_score(model, Frontopariet_mean.iloc[:, 3:81], Frontopariet_mean.iloc[:, 1], scoring='accuracy', cv=cv, n_jobs=-1)

# Report the model performance (M = 0.56, SD = 0.033)
print('Mean Accuracy: %.3f (%.3f)' % (np.mean(n_scores_FP), np.std(n_scores_FP)))

Mean Accuracy: 0.560 (0.033)


In [349]:
# Evaluate the model and collect the scores - Auditory
n_scores_AU = cross_val_score(model, Auditory_mean.iloc[:, 3:81], Auditory_mean.iloc[:, 1], scoring='accuracy', cv=cv, n_jobs=-1)

# Eeport the model performance (M = 0.50, SD = 0.036)
print('Mean Accuracy: %.3f (%.3f)' % (np.mean(n_scores_AU), np.std(n_scores_AU)))

Mean Accuracy: 0.502 (0.036)


In [350]:
# Evaluate the model and collect the scores - Dorsal-atten
n_scores_DA = cross_val_score(model, Dorsal_atten_mean.iloc[:, 3:81], Dorsal_atten_mean.iloc[:, 1], scoring='accuracy', cv=cv, n_jobs=-1)

# Report the model performance (M = 0.45, SD = 0.038)
print('Mean Accuracy: %.3f (%.3f)' % (np.mean(n_scores_DA), np.std(n_scores_DA)))

Mean Accuracy: 0.448 (0.038)


In [351]:
# Evaluate the model and collect the scores - Posterior-Mu
n_scores_PM = cross_val_score(model, Posterior_Mu_mean.iloc[:, 3:81], Posterior_Mu_mean.iloc[:, 1], scoring='accuracy', cv=cv, n_jobs=-1)

# Report the model performance (M = 0.46, SD = 0.041)
print('Mean Accuracy: %.3f (%.3f)' % (np.mean(n_scores_PM), np.std(n_scores_PM)))

Mean Accuracy: 0.459 (0.041)


In [352]:
# Evaluate the model and collect the scores - Orbito-Affec
n_scores_OA = cross_val_score(model, Orbito_Affec_mean.iloc[:, 3:81], Orbito_Affec_mean.iloc[:, 1], scoring='accuracy', cv=cv, n_jobs=-1)

# Report the model performance (M = 0.37, SD = 0.035)
print('Mean Accuracy: %.3f (%.3f)' % (np.mean(n_scores_OA), np.std(n_scores_OA)))

Mean Accuracy: 0.365 (0.035)


In [353]:
# Evaluate the model and collect the scores - Ventral-Mult
n_scores_VM = cross_val_score(model, Ventral_Mult_mean.iloc[:, 3:81], Ventral_Mult_mean.iloc[:, 1], scoring='accuracy', cv=cv, n_jobs=-1)

# Report the model performance (M = 0.49, SD = 0.044)
print('Mean Accuracy: %.3f (%.3f)' % (np.mean(n_scores_VM), np.std(n_scores_VM)))

Mean Accuracy: 0.489 (0.044)


In [413]:
X_2back_full_net.head()

,Subject_id,Condition,Network,0,1,2,3,4,5,6,...,68,69,70,71,72,73,74,75,76,77
0,0,body,Auditory,10513.302667,10510.148000,10535.079333,10521.920000,10515.240667,10529.814667,10548.097333,...,10547.381333,10548.404667,10581.230667,10578.819333,10552.318000,10553.810000,10557.700000,10547.729333,10555.196000,10572.920667
1,0,body,Cingulo-Oper,11359.838393,11350.669643,11362.828036,11357.317143,11354.753036,11354.653036,11372.082679,...,11184.943571,11190.940714,11186.418750,11194.230893,11187.954643,11196.669643,11194.907500,11183.193929,11209.787500,11206.237500
2,0,body,Default,10280.175217,10277.216522,10268.863913,10280.523478,10267.522174,10282.300870,10301.117391,...,10348.343478,10356.838261,10364.836957,10358.145217,10355.730000,10362.967391,10361.825217,10340.093913,10340.578696,10358.893913
3,0,body,Dorsal-atten,10407.442857,10426.668571,10380.267143,10386.244286,10394.920000,10415.598571,10426.321429,...,10369.707143,10384.895714,10395.381429,10389.944286,10396.580000,10404.450000,10411.432857,10394.377143,10414.470000,10411.192857
4,0,body,Frontopariet,10888.065600,10873.332200,10855.890200,10869.319600,10884.365800,10897.929400,10929.934000,...,10657.163000,10687.228600,10686.270600,10684.482400,10675.743600,10698.277000,10689.713600,10680.966000,10680.650800,10679.955400


In [412]:
# Return indices to columns (2-back)
X_2back_full_net.reset_index(level=0, inplace=True)
X_2back_full_net.reset_index(level=0, inplace=True)
X_2back_full_net.reset_index(level=0, inplace=True)

In [414]:
Visual2_2back = X_2back_full_net.loc[X_2back_full_net['Network'] == 'Visual2']

In [415]:
unique(Visual2_2back['Network'])

Visual2


In [416]:
# 2 back

# Evaluate the model and collect the scores - Visual 2
n_scores_V2_2back = cross_val_score(model, Visual2_2back.iloc[:, 3:81], Visual2_2back.iloc[:, 1], scoring='accuracy', cv=cv, n_jobs=-1)

# Report the model performance 
print('Mean Accuracy: %.3f (%.3f)' % (np.mean(n_scores_V2_2back), np.std(n_scores_V2_2back)))

Mean Accuracy: 0.685 (0.043)


#### Note:

To be continued.